# Subduction plots

In [5]:
import os
import numpy
from utils import write_mesh, write_poly
from IPython.display import Image
from IPython.core.display import HTML 
from openquake.sub.misc.profile import _read_profiles
from openquake.sub.misc.edge import create_from_profiles
from openquake.sub.misc.utils import create_inslab_meshes
from openquake.sub.quad.msh import create_lower_surface_mesh

In [6]:
tmp_path = "./tmp"
if not os.path.exists(tmp_path):
    os.mkdir(tmp_path)
    os.mkdir(os.path.join(tmp_path, 'vf'))    

In [7]:
path = "./../model/profiles/"
profiles, _ = _read_profiles(path)
slab_thickness = 50.
sampling = 40
idl = False
alg = False
smsh = create_from_profiles(profiles, sampling, sampling, idl, alg)
low_mesh = create_lower_surface_mesh(smsh, 50)

Reading profiles from ./../model/profiles/cs*.*


ValueError: negative dimensions are not allowed

## Plotting profiles and traces

In [ ]:
%%bash -s "$path" 

catalogue='./../data/catalogue/wpg_catalogue_sumatra.csv'
catalogue='./../data/catalogue/catalogue.csv'

gmt set MAP_FRAME_TYPE = PLAIN
gmt set PS_MEDIA = a4

ext='-R94/107/-6/7/0/200'
prj='-Jm1 -Jz-0.01 -p80/30/0'
out='./tmp/profiles.ps'

DATA=$1'/cs*'
cpt=./tmp/tmp.cpt

gmt psbasemap $ext $prj -K -B+n > $out
gmt pscoast $ext $prj -O -K -Glightbrown >> $out
gmt makecpt -Chaxby -T0/200/20 > $cpt
tail -n+2 $catalogue | gawk -F, '{print $3, $4, $5, $5, $6/80}' | gmt psxyz $ext $prj -O -K -C$cpt -Sc >> $out

for f in $DATA; 
do 
    gmt psxyz $f $ext $prj -O -K -Wthick,green,- >> $out
    gmt psxy $f $ext $prj -O -K -Wthick,red >> $out
done

gmt pscoast $ext $prj -O -K -W0.5 >> $out
gmt psbasemap $ext $prj -O -B+n >> $out
gmt psconvert $out -A0.5c -P -Tg 


rm gmt.conf gmt.history
echo 'done'

done


In [ ]:
Image(url= "./tmp/profiles.png")

## Plotting mesh

First we create a temporary file containing the edges created starting from the profiles.

In [ ]:
edges_fname, profiles_fname = write_mesh(smsh, 'top')
edges_low_fname, profiles_low_fname = write_mesh(low_mesh, 'low')
poly_fname = write_poly(smsh, 'poly')
poly_low_fname = write_poly(low_mesh, 'poly_low')

In [ ]:
%%bash -s "$path" "$edges_fname" "$profiles_fname" "$poly_low_fname" "$poly_fname"

ext='-R94/107/-6/7/0/200'
prj='-Jm1 -Jz-0.01 -p100/20/0'
out='./tmp/edges.ps'
DATA=$1'/cs*'

gmt psbasemap $ext $prj -K -B+b > $out

gmt psxyz $4 $ext $prj -O -K -Wdefault,orange -G240 >> $out
gmt psxyz $5 $ext $prj -O -K -Wthinnest,purple -G220 >> $out

for f in $DATA; 
do 
    gmt psxyz $f $ext $prj -O -K -Wthinner,green,- >> $out
done

gmt psbasemap $ext $prj -O -B+b >> $out
gmt psconvert $out  -A0.5c -P -Tg 
echo 'done'

done


In [ ]:
Image(url= "./tmp/edges.png")

## Plotting virtual faults

In [ ]:
dips = [45, 135]
sampling = 20.
ohs = create_inslab_meshes(smsh, dips, slab_thickness, sampling)

In [ ]:
!mkdir -p tmp/vf

In [ ]:
for key in ohs:
    for i, fault in enumerate(ohs[key]):
        fou = open(os.path.join(tmp_path, 'vf', 'vf_{:d}_{:03d}.txt'.format(key, i)), 'w')
        for pro in fault:
            fou.write('>\n')
            for p in pro:
                fou.write('{:.5f},{:.5f},{:.5f}\n'.format(p.longitude, p.latitude, p.depth))
        fou.write('>\n')
        fou.close()

In [ ]:
sampling = 15.
tmp_mesh = create_from_profiles(ohs[135][4], sampling, sampling, idl, alg)
vf_edges_fname, vf_profiles_fname = write_mesh(tmp_mesh, 'vf')
vf_poly_fname = write_poly(tmp_mesh, 'vf_poly')

In [ ]:
%%bash -s "$path" "$vf_edges_fname"  "$vf_profiles_fname" "$poly_low_fname"  "$poly_fname" "$vf_poly_fname"

ext='-R94/107/-6/7/0/200'
prj='-Jm1 -Jz-0.01 -p220/30/0'
out='./tmp/virtual_faults0.ps'
DATA=$1'cs*'

gmt psbasemap $ext $prj -K -B+b > $out

gmt psxyz $4 $ext $prj -O -K -Wdefault,orange -G240 >> $out
gmt psxyz $5 $ext $prj -O -K -Wthinnest,purple -G220 >> $out
gmt psxyz $6 $ext $prj -O -K -Wthinnest,red -G200 >> $out
gmt psxyz $4 $ext $prj -O -K -Wdefault,orange,. >> $out
gmt psxyz $5 $ext $prj -O -K -Wthinnest,purple >> $out

gmt psxyz ./tmp/vf/vf_135_004.txt $ext $prj -O -K -Wthinner,red >> $out

gmt psbasemap $ext $prj -O -B+n >> $out
gmt psconvert $out -A0.5c -P -Tg 

echo 'done'

done


In [ ]:
Image(url= "./tmp/virtual_faults0.png")

In [ ]:
%%bash -s "$path" "$vf_edges_fname"  "$vf_profiles_fname" "$poly_low_fname"  "$poly_fname" "$vf_poly_fname"

ext='-R94/107/-6/7/0/200'
prj='-Jm1 -Jz-0.01 -p3200/10/0'
out='./tmp/virtual_faults1.ps'
DATA=$1'cs*'

gmt psbasemap $ext $prj -K -B+b > $out

gmt psxyz $4 $ext $prj -O -K -Wdefault,orange -G240 >> $out
gmt psxyz $5 $ext $prj -O -K -Wthinnest,purple -G220 >> $out
gmt psxyz $6 $ext $prj -O -K -Wthinnest,red -G200 >> $out
gmt psxyz $5 $ext $prj -O -K -Wthinnest,purple >> $out

gmt psxyz ./tmp/vf/vf_135_004.txt $ext $prj -O -K -Wthinner,red >> $out

gmt psbasemap $ext $prj -O -B+n >> $out
gmt psconvert $out -A0.5c -P -Tg 
echo 'done'

done


In [ ]:
Image(url= "./tmp/virtual_faults1.png")